## Thư viện 



In [1]:
import pandas as pd
import numpy as np
import pymap3d as pm
from UKF_25_5_2024.quaternion_Madgwick import Madgwick # sử dụng để khai báo hàm tự tạo 
from UKF_25_5_2024.quaternion_KalmanFilter import Kalman
from UKF_25_5_2024.convert_coordinate_system import CCS

## Xử lí IMU và đưa về giá trị trung bình trùng với mốc thời gian GPS

In [3]:
imu_data = pd.read_csv(r'C:\Users\luan1\OneDrive\Desktop\DATN_20232\Code\GNSS-Positioning\data\2020-06-25-00-34-us-ca-mtv-sb-101\pixel4xl\device_imu.csv')
gnss_data = pd.read_csv(r'C:\Users\luan1\OneDrive\Desktop\DATN_20232\Code\GNSS-Positioning\data\2020-06-25-00-34-us-ca-mtv-sb-101\pixel4xl\device_gnss.csv')
# latlong_data = pd.read_csv(r'D:\DATN_20232\Code\GNSS-Positioning\gnss_latlong.csv')

# Khởi tạo danh sách để lưu kết quả trung bình
result = []

# Vòng lặp qua từng giá trị thời gian trong file GNSS
for epoch in gnss_data['utcTimeMillis'].unique():
    # Lọc dữ liệu IMU theo khoảng thời gian 1 giây tương ứng với giá trị thời gian GNSS
    last_three_digits = epoch % 1000
    epoch_start = epoch - last_three_digits
    epoch_end = epoch_start + 1000
    epoch_data = imu_data[(imu_data['utcTimeMillis'] >= epoch_start) & (imu_data['utcTimeMillis'] < epoch_end)]
    
    if not epoch_data.empty:
        # Tính giá trị trung bình cho các dữ liệu Accel, Gyro, Mag
        mean_accel = epoch_data[epoch_data['MessageType'] == 'UncalAccel'][['MeasurementX', 'MeasurementY', 'MeasurementZ']].mean()
        mean_gyro = epoch_data[epoch_data['MessageType'] == 'UncalGyro'][['MeasurementX', 'MeasurementY', 'MeasurementZ']].mean()
        mean_mag = epoch_data[epoch_data['MessageType'] == 'UncalMag'][['MeasurementX', 'MeasurementY', 'MeasurementZ']].mean()
        mean_bias_mag = epoch_data[epoch_data['MessageType'] == 'UncalMag'][['BiasX', 'BiasY', 'BiasZ']].mean()
        
        #Tạo dictionary lưu kết quả cho epoch hiện tại
        result.append({
            'utcTimeMillis': epoch,
            'mean_accel_x': mean_accel['MeasurementX'],
            'mean_accel_y': mean_accel['MeasurementY'],
            'mean_accel_z': mean_accel['MeasurementZ'],
            'mean_gyro_x': mean_gyro['MeasurementX'],
            'mean_gyro_y': mean_gyro['MeasurementY'],
            'mean_gyro_z': mean_gyro['MeasurementZ'],
            'mean_mag_x': mean_mag['MeasurementX'],
            'mean_mag_y': mean_mag['MeasurementY'],
            'mean_mag_z': mean_mag['MeasurementZ'],
            'mean_bias_mag_x': mean_bias_mag['BiasX'],
            'mean_bias_mag_y': mean_bias_mag['BiasY'],
            'mean_bias_mag_z': mean_bias_mag['BiasZ']
        })
        
'''
    Lưu DataFrame vào file CSV
'''
# Chuyển danh sách kết quả thành DataFrame
result_df = pd.DataFrame(result)
print(result_df.head())

# Xuất kết quả ra file CSV
output_file = 'imu_averaged_original.csv'
result_df.to_csv(output_file, index=False)
print(f"Saved done {output_file}")


   utcTimeMillis  mean_accel_x  mean_accel_y  mean_accel_z  mean_gyro_x  \
0  1593045251447     -0.574866      9.791248      0.578658     0.006419   
1  1593045252447     -0.506329      9.808715      0.555514    -0.000546   
2  1593045253447     -0.527709      9.810021      0.565149    -0.000141   
3  1593045254447     -0.527479      9.813017      0.549107     0.001783   
4  1593045255447     -0.992141      9.875990     -0.535992    -0.001148   

   mean_gyro_y  mean_gyro_z  mean_mag_x  mean_mag_y  mean_mag_z  \
0    -0.010799     0.001549   -0.687723  -56.013615   -7.553571   
1     0.000758    -0.000399   -1.025000  -56.025378   -7.599621   
2     0.001275    -0.000455   -1.227934  -56.250382   -7.385969   
3    -0.000364    -0.000426   -1.359470  -56.071590   -7.169697   
4     0.048918     0.001742   -1.520455  -56.460984   -6.720833   

   mean_bias_mag_x  mean_bias_mag_y  mean_bias_mag_z  
0        31.420649       -34.034336       -32.969463  
1        31.420649       -34.034336 

## Quaternion và chuyển hệ trục body(B) về hệ trục ECEF

In [6]:
# Đọc dữ liệu từ tệp CSV
csv_file_path = "imu_averaged_original.csv"
data = pd.read_csv(csv_file_path)

# Tạo đối tượng Madgwick và CCS
filter = Madgwick()

# Xử lý dữ liệu và lưu các giá trị roll, pitch, yaw, accel và gyro
results_ecef = []
results_ned = []

for index, row in data.iterrows():
    filter.updateRollPitchYaw(
        row["mean_accel_x"], row["mean_accel_y"], row["mean_accel_z"],
        row["mean_gyro_x"], row["mean_gyro_y"], row["mean_gyro_z"],
        row["mean_mag_x"], row["mean_mag_y"], row["mean_mag_z"], dt=0.1
    )
    
    # Chuyển đổi từ body sang ECEF
    phi, theta, psi = filter.roll, filter.pitch, filter.yaw
    ccs = CCS(phi, theta, psi)
    accel_ecef = ccs.body_to_ecef(
        row["mean_accel_x"], row["mean_accel_y"], row["mean_accel_z"]
    )
    gyro_ecef = ccs.body_to_ecef(
        row["mean_gyro_x"], row["mean_gyro_y"], row["mean_gyro_z"]
    )
    accel_ned = ccs.body_to_ned(
        row["mean_accel_x"], row["mean_accel_y"], row["mean_accel_z"]
    )
    gyro_ned = ccs.body_to_ned(
        row["mean_gyro_x"], row["mean_gyro_y"], row["mean_gyro_z"]
    )



    # print(f"utcTimeMillis: {row['utcTimeMillis']}, Roll: {filter.roll}, Pitch: {filter.pitch}, Yaw: {filter.yaw}")
    # print(f"Accelerometer ECEF: {accel_ecef}, Gyroscope ECEF: {gyro_ecef}")

    results_ecef.append({
        "utcTimeMillis": row["utcTimeMillis"],
        "accel_ecef_x": accel_ecef[0],
        "accel_ecef_y": accel_ecef[1],
        "accel_ecef_z": accel_ecef[2],
        "gyro_ecef_x": gyro_ecef[0],
        "gyro_ecef_y": gyro_ecef[1],
        "gyro_ecef_z": gyro_ecef[2]
    })

    results_ned.append({
        "utcTimeMillis": row["utcTimeMillis"],
        "accel_ned_x": accel_ned[0],
        "accel_ned_y": accel_ned[1],
        "accel_ned_z": accel_ned[2],
        "gyro_ned_x": gyro_ned[0],
        "gyro_ned_y": gyro_ned[1],
        "gyro_ned_z": gyro_ned[2]
    })

# Chuyển đổi dữ liệu đã xử lý thành DataFrame
output_df = pd.DataFrame(results_ecef)
# Đường dẫn tệp CSV đầu ra
output_csv_file_path = "imu_ecef.csv"
# Xuất dữ liệu ra tệp CSV
output_df.to_csv(output_csv_file_path, index=False)
# Hiển thị vài dòng đầu tiên của kết quả để kiểm tra
print(output_df.head())

# Chuyển đổi dữ liệu đã xử lý thành DataFrame
output_df = pd.DataFrame(results_ned)
# Đường dẫn tệp CSV đầu ra
output_csv_file_path = "imu_ned.csv"
# Xuất dữ liệu ra tệp CSV
output_df.to_csv(output_csv_file_path, index=False)
# Hiển thị vài dòng đầu tiên của kết quả để kiểm tra
print(output_df.head())


   utcTimeMillis  accel_ecef_x  accel_ecef_y  accel_ecef_z  gyro_ecef_x  \
0   1.593045e+12      0.203092      9.818055     -0.313683     0.005564   
1   1.593045e+12      0.989346      9.706824     -1.254834    -0.000436   
2   1.593045e+12      1.610228      9.464647     -2.159167     0.000114   
3   1.593045e+12      2.166852      9.098434     -3.065667     0.001585   
4   1.593045e+12      2.111112      8.359995     -4.945756     0.014425   

   gyro_ecef_y  gyro_ecef_z  
0    -0.011089     0.002510  
1     0.000747    -0.000533  
2     0.001102    -0.000790  
3    -0.000969    -0.000206  
4     0.042134    -0.020345  
   utcTimeMillis  accel_ned_x  accel_ned_y  accel_ned_z  gyro_ned_x  \
0   1.593045e+12     0.203092     9.818055    -0.313683    0.005564   
1   1.593045e+12     0.989346     9.706824    -1.254834   -0.000436   
2   1.593045e+12     1.610228     9.464647    -2.159167    0.000114   
3   1.593045e+12     2.166852     9.098434    -3.065667    0.001585   
4   1.593045e+